# Encrypted Inference - Multi-Party Computation

In [1]:
%install-location $cwd/swift-install
%install '.package(url: "https://github.com/tensorflow/swift-models", .branch("master"))' Datasets
%install '.package(path: "$cwd/ppmlNB_encrypted_tensor")' ppmlNB_encrypted_tensor

Installing packages:
	.package(url: "https://github.com/tensorflow/swift-models", .branch("master"))
		Datasets
	.package(path: "/root/swift-ppml/secure_computation/ppmlNB_encrypted_tensor")
		ppmlNB_encrypted_tensor
With SwiftPM flags: []
Working in: /tmp/tmpnpi8qsm3/swift-install
[1/2] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[2/3] Merging module jupyterInstalledPackages
Initializing Swift...
Installation complete!


In [2]:
import ppmlNB_encrypted_tensor
import TensorFlow

## Introduction

In this example, we are performing inferences on encrypted images of digits using multi-party computation protocol. The idea is to split values into shares among several parties (for example 2 servers). One key property of this protocol is that the shares owned by the different parties don't reveal anything about the original values. However what's beautiful about this cryptographic technique is that we can still compute functions (e.g., matmul, add, etc.) on it. The original values get revealed only if the parties decide to combine their shares. MPC assumes that these parties won't collude. However they accept to perform some computations together (e.g., inference on images). 

You can learn more about encrypted machine learning with MPC from this [fantastic blog post](https://mortendahl.github.io/2017/04/17/private-deep-learning-with-mpc/).

In this notebook, we will first publicly train a model classifying images of digits from the MNIST dataset. By public training, we mean that the images and model are not encrypted. Once the model is trained, we will encrypt the images and the model and try to classify them even though they are encrypted :).

## Step1: Public Training

In the code below, we train a model to classify images of digits (0 to 9). For the model, we are using a basic logistic regression. For this toy example, we just implemented MatMul and Add in MPC. However, we could extend the protocol to more complex neural net layers such as Conv2d.

In [3]:
import Datasets

let epochCount = 12
let batchSize = 128

let dataset = MNIST()

var classifier = Sequential {
    Flatten<Float>()
    Dense<Float>(inputSize: 784, outputSize: 10)
}

let optimizer = SGD(for: classifier, learningRate: 0.1)

print("Beginning training...")

struct Statistics {
    var correctGuessCount: Int = 0
    var totalGuessCount: Int = 0
    var totalLoss: Float = 0
    var batches: Int = 0
}

let testBatches = dataset.testDataset.batched(batchSize)

// The training loop.
for epoch in 1...epochCount {
    var trainStats = Statistics()
    var testStats = Statistics()
    let trainingShuffled = dataset.trainingDataset.shuffled(
        sampleCount: dataset.trainingExampleCount, randomSeed: Int64(epoch))

    Context.local.learningPhase = .training
    for batch in trainingShuffled.batched(batchSize) {
        let (labels, images) = (batch.label, batch.data)
        // Compute the gradient with respect to the model.
        let 𝛁model = TensorFlow.gradient(at: classifier) { classifier -> Tensor<Float> in
            let ŷ = classifier(images)
            let correctPredictions = ŷ.argmax(squeezingAxis: 1) .== labels
            trainStats.correctGuessCount += Int(
                Tensor<Int32>(correctPredictions).sum().scalarized())
            trainStats.totalGuessCount += batchSize
            let loss = softmaxCrossEntropy(logits: ŷ, labels: labels)
            trainStats.totalLoss += loss.scalarized()
            trainStats.batches += 1
            return loss
        }
        // Update the model's differentiable variables along the gradient vector.
        optimizer.update(&classifier, along: 𝛁model)
    }

    Context.local.learningPhase = .inference
    for batch in testBatches {
        let (labels, images) = (batch.label, batch.data)
        // Compute loss on test set
        let ŷ = classifier(images)
        let correctPredictions = ŷ.argmax(squeezingAxis: 1) .== labels
        testStats.correctGuessCount += Int(Tensor<Int32>(correctPredictions).sum().scalarized())
        testStats.totalGuessCount += batchSize
        let loss = softmaxCrossEntropy(logits: ŷ, labels: labels)
        testStats.totalLoss += loss.scalarized()
        testStats.batches += 1
    }

    let trainAccuracy = Float(trainStats.correctGuessCount) / Float(trainStats.totalGuessCount)
    let testAccuracy = Float(testStats.correctGuessCount) / Float(testStats.totalGuessCount)
    print(
        """
        [Epoch \(epoch)] \
        Training Loss: \(trainStats.totalLoss / Float(trainStats.batches)), \
        Training Accuracy: \(trainStats.correctGuessCount)/\(trainStats.totalGuessCount) \
        (\(trainAccuracy)), \
        Test Loss: \(testStats.totalLoss / Float(testStats.batches)), \
        Test Accuracy: \(testStats.correctGuessCount)/\(testStats.totalGuessCount) \
        (\(testAccuracy))
        """)
}

Loading resource: train-images-idx3-ubyte
Loading resource: train-labels-idx1-ubyte
2020-02-09 21:49:18.407220: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 188160000 exceeds 10% of system memory.
2020-02-09 21:49:18.461554: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2020-02-09 21:49:18.474753: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2208000000 Hz
2020-02-09 21:49:18.476603: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xde24f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-02-09 21:49:18.476652: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-02-09 21:49:18.478727: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 188160000 exceeds 10% of system memory.
Loading resource: t10

As you can see, this basic model acheives 91.1% accuracy on the testing set. We could definitely improve the accuracy with a more complex model.

## Step 2: Save Model Weights

Once the model is trained, let's save its weights to later perform encrypted inferences.

In [4]:
// Extract weights
let w1 = classifier.layer2.weight
let b1 = classifier.layer2.bias

Here we just extract several images from the MNIST dataset.

In [5]:
func get_input(_ nb_input: Int) -> (Tensor<Float>, Tensor<Int32>){
    let dataset = MNIST()
    let trainingShuffled = dataset.trainingDataset.shuffled(
        sampleCount: dataset.trainingExampleCount, 
        randomSeed: Int64(1)).batched(nb_input)
    var train_iter = trainingShuffled.self.makeIterator()
    var image = train_iter.next().unsafelyUnwrapped
    return (image.data.reshaped(to: [nb_input, 784]), image.label)
    
}

In [6]:
let (input, label) = get_input(3)

Loading resource: train-images-idx3-ubyte
Loading resource: train-labels-idx1-ubyte
2020-02-09 21:49:50.357843: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 188160000 exceeds 10% of system memory.
2020-02-09 21:49:50.434752: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 188160000 exceeds 10% of system memory.
Loading resource: t10k-images-idx3-ubyte
Loading resource: t10k-labels-idx1-ubyte


## Step3: Encrypted Inference

Awesome! Our model is ready, so let's perform some private predictions. To encrypt an image, we can convert the tensor into a PrivateTensor as follow:

In [7]:
let encryped_images = PrivateTensor.from_values(values: input)

In [8]:
encryped_images

▿ PrivateTensor
  - share0 : [[-6787589838563964911,  6448897555809465334,  7934375839428403173, ...,
   8672702744847897347, -7765916615583691251,  1436726397468966556],
 [ 3863737563595048021,   -15318943555949504,  6405016834740266553, ...,
  -4339131960208094544, -4073941091161145433,  6667856077130869403],
 [-4396048434353189395, -2984543475000313945, -4412639684622742885, ...,
  -7981350163169210170,  3228171541547781169,  5744873139302189729]]
  - share1 : [[ 6787589838563964911, -6448897555809465334, -7934375839428403173, ...,
  -8672702744847897347,  7765916615583691251, -1436726397468966556],
 [-3863737563595048021,    15318943555949504, -6405016834740266553, ...,
   4339131960208094544,  4073941091161145433, -6667856077130869403],
 [ 4396048434353189395,  2984543475000313945,  4412639684622742885, ...,
   7981350163169210170, -3228171541547781169, -5744873139302189729]]


As you can see above, once the original tensor (image) is transformed into a PrivateTensor, its values are split into shares among two parties. Again one of the key property of MPC is that these shares are not revealing about the original values. It would be impossible for each party if the image is a 1 or 3 or 9 etc. For this towy example, the shares are on the same machine, but in practice they could be on different servers. 

Now we compute our logistic regression on the encrypted images. It's just a MatMul and an addition.

In [9]:
func private_prediction(input: Tensor<Float>) -> PrivateTensor{
    let input_private = PrivateTensor.from_values(values: input)
    let w1_private = PrivateTensor.from_values(values: w1)
    let b1_private = PrivateTensor.from_values(values: b1)
    let private_pred = add(x: matmul(x: input_private, y: w1_private), y: b1_private)
    return private_pred
}

Note that we are also encrypting the model (`PrivateTensor.from_values`). So if we deploy this model into the cloud, even the model would be encrypted.

Now, let's perform some private predictions!

In [10]:
for i in 0...2 {
    var private_output = private_prediction(input: input[i].reshaped(to: [1,784]))
    var decode_prediction = private_output.reveal().decode()
    var private_class = decode_prediction.argmax(squeezingAxis: 1)[0]
    var exp_class = label[i]
    print("Private prediction result: \(private_class). Expected prediction: \(exp_class)")
}

Private prediction result: 0. Expected prediction: 0
Private prediction result: 5. Expected prediction: 5
Private prediction result: 1. Expected prediction: 1


As you can see above, we are abale to perform predictions on enrypted data with an encrypted model. At no point during this computation are we decrypting these values.